### Spark의 데이터 처리 아키텍처
1. 데이터 처리 : Dataset(RDD 또는 Dataframe)의 변환 과정
- 데이터를 변경하는게 아니라, 새로운 dataset을 생성

2. Spark은 함수형 프로그래밍 언어
- 함수(변환, transformation)의 인자로 함수를 받아들일 수 있음.
        Dataset2 = Dataset1.변환함수1
        Dataset3 = Dataset2.변환함수2 -> Dataset1.변환함수1.변환함수2

### RDD의 종류
- (base) RDD
    - Element(레코드라고도 할 수 있음)들로 구성됨
    - Element
        - int, Double, String 등의 데이터 뿐만 아니라
          Tuple 일 수도 있고, Array 일수도 있음.
          (string, csv 등을 split 하면 Array가 만들어짐)

- Pair RDD
    - (key, value) -> 위치에 의하여 정해진다
- Multi-column RDD
    - ([0], [1], [2], ...)
- DataFrame
    - (colname1, colname2, ..., colnameN)

### RDD 생성

- parallelizing an exisiting collection in your driver program,
> numRDD = sc.parallelize([1,2,3,4,5])

- referencing a dataset in an external storage system
    - a shared file system
    - HDFS
    - HBase
    - any data source offering a Hadoop input Format
> lines = sc.textFile("C://spark/README.md")

In [1]:
import findspark
findspark.init()
import pyspark

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").getOrCreate()

sc = spark.sparkContext

In [3]:
numRDD = sc.parallelize([1, 2, 3, 4, 5, 6])

In [4]:
numRDD.count()

6

In [5]:
type(numRDD)

pyspark.rdd.RDD

3. 지연 실행 (lazy evaluation)
- 데이터 변환 함수(transformation)를 곧바로 실행하지 않음
    - 변형 방법만 기록해 둠.
    - 데이터 출력이나 저장 활동이 일어날 때 기억해 둔 모든 변환 작업을 한꺼번에 실행함.
- 프로그램 개발 과정에서는 각 변환 함수를 실행할 때마다, 출력 등의 활동(Action)을
  실행해서 입력된 변환 명령어가 오류가 없는지를 확인해보는 것이 좋음.
     - 완성된 프로그램은 함수형으로 표현하는 것이 간결함.

In [6]:
lines = sc.textFile("C://spark/README.md")

In [7]:
import re

words = lines.flatMap(lambda line: re.split('\\W+', line)) \
             .filter(lambda word: len(word) > 0) \
             .map(lambda word: word.lower()) \
             .map(lambda word: (word, 1))

In [8]:
words

PythonRDD[4] at RDD at PythonRDD.scala:53

### Spark transformations : Basic Operations


#### 개별 원소들을 1:1로 매핑시킨다.
- map(func)
    - 데이터셋의 각 원소들에게 func를 적용한 새로운 데이터셋을 생성
    > wordAndOne = words.map(lambda x: (x,1))

#### 함수
- def 함수명(인자명: 인자타입) -> 결과타입
    - Python은 변수의 유형(type)을 지정하지 않아도 되며, 지정해도 확인하지 않음
    - Default 값을 인자에 부여할 수 있음


#### Spark에서 사용자가 정의한 함수의 사용
- 변환함수의 인자로 사용자정의함수를 사용하면, Dataset의 모든 원소에 대하여
  사용자 정의함수를 적용하는 변환이 이루어짐.


In [9]:
def mySquare(x: float) -> float:
    return x * x

In [10]:
numSquare = numRDD.map(mySquare)

In [11]:
for e in numSquare.collect(): # collect() : RDD를 파이썬의 list로 바꿔주는 함수
    print(e)

1
4
9
16
25
36


#### 익명 함수
- 일반적인 함수: 실명함수
    - 이름을 부여하고
    - 입력 변수를 지정하고
    - 결과값을 돌려받음
- 익명 함수
    - 함수의 이름을 부여하지 않고,
    - 입력 변수와
    - 함수식만을 정의해서 결과값을 돌려받음
- 표시방법
    - Python
        - lambda
    - Scala
        - =>

In [12]:
numSquare2 = numRDD.map(lambda x: x*x)

In [13]:
numSquare2.collect()

[1, 4, 9, 16, 25, 36]

- flatMap(func)
    - map과 비슷하나, 데이터셋의 각 원소들을 여러 개의 원소로 매핑함.
      (따라서 func은 한개의 원소가 아니라 여러 원소들의 연속체를 만들어 냄.)

In [14]:
words2 = lines.flatMap(lambda line: line.split(" "))

- filter(func)
    - func을 실행한 값이 참(True)인 원소들만을 선택하여 새로운 데이터셋을 생성
    - returns true.
    > words = words.filter(lambda w: len(w) > 0)

### RDD Actions

- collect()
    - 모든 원소들의 파이썬 리스트를 생성
- count()
    - 원소들의 개수를 출력
- first()
    - 첫번째 원소를 출력
- take(n)
    - 처음 n개의 원소들을 포함하는 리스트를 생성
- top(n)
    - Return the top n elements from an RDD.

In [15]:
lines.take(3)

['# Apache Spark',
 '',
 'Spark is a unified analytics engine for large-scale data processing. It provides']

In [16]:
lines.top(3)

['will run the Pi example locally.',
 'supports general computation graphs for data analysis. It also supports a',
 'storage systems. Because the protocols have changed in different versions of']

In [17]:
lines.collect()

['# Apache Spark',
 '',
 'Spark is a unified analytics engine for large-scale data processing. It provides',
 'high-level APIs in Scala, Java, Python, and R, and an optimized engine that',
 'supports general computation graphs for data analysis. It also supports a',
 'rich set of higher-level tools including Spark SQL for SQL and DataFrames,',
 'MLlib for machine learning, GraphX for graph processing,',
 'and Structured Streaming for stream processing.',
 '',
 '<https://spark.apache.org/>',
 '',
 '[![Jenkins Build](https://amplab.cs.berkeley.edu/jenkins/job/spark-master-test-sbt-hadoop-2.7-hive-2.3/badge/icon)](https://amplab.cs.berkeley.edu/jenkins/job/spark-master-test-sbt-hadoop-2.7-hive-2.3)',
 '[![AppVeyor Build](https://img.shields.io/appveyor/ci/ApacheSoftwareFoundation/spark/master.svg?style=plastic&logo=appveyor)](https://ci.appveyor.com/project/ApacheSoftwareFoundation/spark)',
 '[![PySpark Coverage](https://img.shields.io/badge/dynamic/xml.svg?label=pyspark%20coverage&url=ht

In [18]:
words2 = lines.flatMap(lambda line: line.split(" "))

In [19]:
words2.take(10)

['#',
 'Apache',
 'Spark',
 '',
 'Spark',
 'is',
 'a',
 'unified',
 'analytics',
 'engine']

In [20]:
import re

In [27]:
words = lines.flatMap(lambda line: re.split("\W+", line))

In [28]:
words.take(3)

['', 'Apache', 'Spark']

In [29]:
words = words.filter(lambda word: len(word) > 0)

In [33]:
words.take(3)

['Apache', 'Spark', 'Spark']

In [35]:
def addOne(x):
    return (x, 1)

In [36]:
wordAndOne = words.map(addOne)

In [37]:
wordAndOne.take(3)

[('Apache', 1), ('Spark', 1), ('Spark', 1)]

In [38]:
type(wordAndOne)

pyspark.rdd.PipelinedRDD

- reduceByKey(func, [numPartitions])
    - (K, V) 쌍의 데이터셋에서 키값이 같은 원소들에 대해서 각 원소의 값에 func를 실행한 데이터를 생성.

In [40]:
wordCount = wordAndOne.reduceByKey(lambda v1, v2: v1 + v2)

In [41]:
wordCount.take(3)

[('Apache', 2), ('Spark', 17), ('is', 7)]

- sortByKey(ascending=True,[numPartitions])
    - (K, V)쌍의 데이터셋의 원소들을 키값의 순서에 따라 정렬한 데이터셋을 생성

In [42]:
sortedWC = wordCount.sortByKey(False)

In [43]:
sortedWC.take(3)

[('your', 1), ('you', 4), ('yarn', 2)]

- sortBy(keyfunc,[ascending],[numPartitions])
    - (K, V) 쌍의 데이터셋의 원소들을 keyfunc 함수의 순서에 따라 정렬한 데이터셋을 생성

In [48]:
sortedWC2 = wordCount.sortBy(lambda x: x[1], False)

In [50]:
sortedWC2.take(20)

[('the', 24),
 ('spark', 21),
 ('Spark', 17),
 ('https', 16),
 ('to', 16),
 ('run', 13),
 ('for', 12),
 ('and', 10),
 ('a', 9),
 ('apache', 9),
 ('org', 9),
 ('is', 7),
 ('on', 7),
 ('html', 7),
 ('example', 7),
 ('can', 6),
 ('building', 6),
 ('000', 6),
 ('1000', 6),
 ('tests', 6)]